# ENG - FORECASTING FUNCTION

_make_forecast2 (df,
                forecast_from, # INCLUDED
                forecast_to, # INCLUDED
                freq = 'D',

                ** kwargs:
                # preprocessing
                preprocess_from, # INCLUDED
                preprocess_to, # INCLUDED
                ds, # column of the df to be renamed to ds if it is not
                target = kwargs.get ('target') # df column to rename to y in case it is not

                # Prophet args
                growth = kwargs.get ('growth', 'linear')
                cap = kwargs.get ('cap')
                floor = kwargs.get ('floor')
                lockdowns = kwargs.get ('lockdowns') # one off holidays

                # additional regressors what if
                regressors = kwargs.get ('regressors') # dictionary pair of regressors and floats to multiply all the values ​​of the regressor
                replace_regressors_values ​​= kwargs.get ('replace_regressors_values') # must be a nested disctionary

                # additionnal regressors to and forcasted
                to_forecast = kwargs.get ('to_forecast') # list like
                mode = kwargs.get ('mode')
                
                # layout
                title = kwargs.get ('title')
                ) _

## 1. MINIMUM INPUTS OLIGATORS

This section will describe the munimi inputs necessary for the function to carry out the forecast.

### 1.1 DATA FRAME
The data frame is the general name with which the database in the form of a relational table is indicated, in python this is obtained through the Pandas library.

#### 1.1.2 MONOVARIATE ANALYSIS
The input dataframe for the single-variant forecast needs at least 2 columns:
- **time:**
the granularity of the dataset depends on this column, the function accepts 2 types of granularity: daily and monthly.
Ideally the column name should be 'ds', however an argument is provided in the function to rename it according to Prophet.
The column dtype must be of the datetime type, if the column is not of this data type, please refer to the pandas documentation at the following link:
https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
- **target:** the target is the column we want to predict, ideally the column name should be 'y', however, similar to the time column, an argument is set up in the function to rename it according to Prophet. The column dtype must necessarily be int or float, if not: df ['col']. Astype (float). The forecast must be made by brand and by region / country.

#### 1.1.2 MULTIVARIATE ANALYSIS

For the multivariate analysis, in addition to the two columns mentioned above for the monovariable forecast, it will be necessary to add a column for each regressor that you want to use for this type of forecast. It is of primary importance that the regressors report only and exclusively the values ​​relating to the current analysis season. If there are missing or non-inherent values, it will be necessary to reset them while maintaining the historical linearity unchanged (no time gaps).

Multivariate analysis can be of three types:
- **multivariate what if:** the future values ​​of the regressors can be modified by the analyst according to the expected values. This replacement can take place either by entering the values ​​relating to the previous season leaving them unchanged or by acting on them with percentage changes applied to the entire forecast period. At the same time an argument of the function has been prepared to allow the variation of the single value of a quadrant (row, column), more in detail in the KWARGS paragraph.
- **multivariate forecast**: if the regressors are not dependent on intrinsic factors and therefore dependent on the will of the management, it will be necessary to predict them to allow the multivariate forecast. Also for this type of analysis a function argument has been prepared to allow this, further detailed in the KWARGS paragraph.
- **hybrid**: what if + forecast

#### 1.2 FORECAST FROM

Forecast from is the second mandatory _positional argument_ to be included in the function. It must be indicated in the form of a string and must be consistent with the date format used in the data stamp column of the input data frame. As the name suggests it tells the function the date from which to start making the forecast, depending on when this falls relative to the reference data frame, the function will output different cases:
- **forecast_from present date and data frame**: testing case. This is the case when you want to test the accuracy of the model. This is done by comparing the output data of the forecast with those actually occurring. To quantify the result, a series of KPIs are returned which will be described in section 1.3.
- **forecast_from = to the day / month following the last reported in the data frame**: operational forecast type case, the outputs will relate to the entire reference period between forecast_from and forecast_to. Depending on when the latter falls, the comparison period of the forecast will be the most recent season ended.
- **forecast_from> of the day / month following the last reported in the data frame**: type of operational forecast, the outputs will only relate to the entire reference period between forecast_from and forecast_to. Depending on when the latter falls, the comparison period of the forecast will be the most recent season ended. Technical detail: dictated by the functioning of Prophet, the real forecast occurs in any case starting from the first day not present in the input data frame, however the KPIs and the output visualization date will only relate to the reference period.

#### 1.3 FORECAST TO

The last positional argument to be entered is forecast_to. As the name suggests it is the date until which the forecast will be made, it will be included in the range. Similarly to forecast_from, also in this case the format must be consistent with the column of data stamp. Depending on when this falls respectively to the input data frame, different scenarios will open:
- **forecast_to within the next year of the input dataset**: if the forecast does not exceed the duration of one year from the last date reported in the input data frame, the forecast will be compared with the previous reference season.
- **forecast_to beyond the next year of the input dataset**: if the forecast exceeds the duration of one year from the last date reported in the input data frame, the forecast will be compared with the reference season of two years earlier. This is done to ensure greater uniformity in the comparison in order to compare only monthly payments relating to the same macro period or season.

## 2. KWARGS

KWARGS are optional arguments of the function. Unlike positional arguments, they do not have to be entered in a precise order, however they need to be preceded by their identifying keyword.

#### 2.1 PREPROCESSING:

The following KWARGS 
Risultati della traduzione
will affect the input data frame:
- **preprocess_from**: date from which you want the input data frame to start. _N.B. the date format must be the same as that used in the data frame, the indicated date is included in the data frame_
- **preprocess_to**: date from which you want the input data frame to end. _N.B. the date format must be the same as that used in the data frame, the indicated date is included in the data frame_
- **ds**: to indicate in the form of a string the name of the column containing the data stamp. This column will be renamed 'ds' as required by Prophet.
- **target**: to indicate the name of the forecast target column in the form of a string. This column will be renamed to 'y' as requested by Prophet.

#### 2.2 PROPHET

The following KWARGS will affect Prophet parameters:
- **freq**: _def. 'M', dtype: str_ indicates the frequency / granularity of the forecast and the input data frame. It can be set to 'M' for monthly forecast or 'D' for daily forecast
- **growth**: _def. 'linear', dtype: str_ indicates the type of growth of the model, 'linear' indicates linear growth, 'logistic' logistics, for the latter it will be necessary to indicate also _cap_ and _floor_.
- **cap**: _dtype: int, float_ upper bound of the forecast
- **floor**: _dtype: int, float_ lower bound of the forecast
- **lockdowns**: _dtype: str_ name of the data frame containing the period (s) to be reported as one-off-holiday: https://facebook.github.io/prophet/docs/handling_shocks.html#treating- covid-19-lockdowns-as-a-one-off-holidays

#### 2.3 ADDITIONAL REGRESSORS FOR WHAT IF ANALYSIS:
- **regressors**: _dtype: dictionary es. {'reg1': 1.0, 'reg2': 1.15} _ must be a dictionary containing as keys the names of the exogenous regressor (s) to be added to the model. The dictionary keys must be columns of the input data frame. The corresponding values ​​are the factors to be multiplied by each value relative to the previous period of the reference regressor. If forecast_to is one year after the last date reported in the data frame, the regressor values ​​will be those of the same time interval as the forecast but two years earlier, otherwise they will refer to the previous year.
- **replace_regressors_values​​**: _dtype: nested dictionary ex. {'reg1': {'data1': val1, 'data2': val2}, 'reg1': {'data1': val1, 'data2': val2}} _ allows to modify the single values ​​relative to that regressor for that specific date. The primary key of the dictionary must be a string indicating the name of the color for which you want to replace the value. The date must be present in the data frame and must be written in the same format as the data frame, otherwise the replacement will not take place.

#### 2.4 ADDITIONAL REGRRSSORS TO BE FORECASTED IN THEIR TIME

If there are exogenous regressors whose future values ​​cannot be estimated or imputed by the user, it will be necessary that these in turn be forecast. At the moment the forecast takes place in a univariate autoregressive manner for each of them. In the future, the possibility of implementing the possibility of multivariate forecasts on them should not be excluded. The presence of this keword argument will add to the outputs the lineplot of the forecasted regressor compared to the real data of the same reference period (in the case of a test case) or compared to the data of one or two previous years depending on whether forecast_to is of one or more years after the last date reported in the input data frame.

- **to_forecast**: _dtype: list_ containing strings containing the names of the columns relating to the regressors to be forecasted to be added to the multivariate analysis.
- **mode**: _dtype: string_ mode can be either 'additive' (default) or 'multiplicative', the kwargs will affect all regressors.

#### 2.5 LAYOUT

Arguments for changing the layout of the output graphs

- **title**: used to modify the title of the comparison barplot between forecast and real data

## 3. OUTPUT

The outputs of the function vary according to the specific forecast, however there are still data and KPIs that will always be provided in output.

### 3.1 COMMON OUTPUT

- **tot forecast**: aggregate value of the target in the reference period.
- **comparison value**: value to have a comparison between a data actually observed and the forecast value.
- **delta real and forecast value**: expressed both in real and percentage terms.
- **pivot table**: showing the aggregate data by month in row and by year in column.
- **forecast table**: showing forecast data with lower and upper bounds expressed both in monetary and percentage terms.
- **pivot displayed with plotly**
- **lineplot**: showing the forecast data (thin blue line), light blue band of lower and upper bounds, real data reported with black dots.
- **bar plot**: showing the forecast data compared with real data.
- **components plot**: showing thirty annually, monthly, weekly and in the event that there is a contribution from the regressors.

### 3.2 OUTPUT WHAT IF

- **additional regressors**: list of exogenous regressors used in the multivariate forecast with related multiplication factors.
- **exogenous regressor data frame**: data frame showing all the values ​​used for the multivariate forecast.

### 3.3 FORECAST MULTIVARIATE OUTPUT

- **regressor lineplot**: lineplot for displaying the regressor univariate forecast to be used for the multivariate forecast.

### 3.4 OUTPUT TEST

- **The Root Mean Squared Error**: average value of the absolute deviation of the forecast from the real value for the period in question, obtained by raising the error to the second. This measure penalizes higher deviations in absolute value.
- **The Mean Absolute Error**: average value of the absolute deviation of the forecast from the real value for the period in question, obtained by averaging the absolute value of the error. This measurement does not penalize the magnitude of the error. _N.B. the smaller the difference between RMSE and MAE, the more it means that the error will be smaller than the total value of the forecast over the entire period_
- **The OVERALL MAPE**: average of the absolute deviations of the forecast from the real values ​​of each forecast point.
- **The OVERALL WAPE**: sum of the losses of the single period on the real value of the entire period.
- **The OVERALL WAPE MEAN**: average of the losses of the single period on the real value of the entire period.
- **additional regressors**: list of exogenous regressors used in the multivariate forecast with related multiplication factors.
- **exogenous regressor data frame**: data frame showing all the values ​​used for the multivariate forecast.
- **data frame real values​​**: referred to the entire period covered by the forecast.
- **forecast value data frame**: refers to the entire forecast period with upper and lower bounds.
- **final data frame**: comparison referring to the entire forecast period with upper, lower bound and wape.
- **final data frame plotly**: comparison referring to the entire forecast period with upper, lower bound and wape.
- **lineplot**: showing real data in pink.
- **barplot**: showing real data in orange.
- **components plot**: showing thirty annually, monthly, weekly and in the event that there is a contribution from regressors.
Ulteriori informazioni su questo testo di origine

# ITA - FUNZIONE DI FORECASTING

_make_forecast2(df,
                forecast_from, # INCLUDED
                forecast_to, # INCLUDED
                freq='D',

                **kwargs:
                # preprocessing
                preprocess_from, # INCLUDED
                preprocess_to, # INCLUDED
                ds, # colonna del df da rinominare in ds nel caso in cui non lo fosse
                target = kwargs.get('target') # colonna del df da rinominare in y nel caso in cui non lo fosse

                # Prophet args
                growth = kwargs.get('growth', 'linear')
                cap  = kwargs.get('cap')
                floor  = kwargs.get('floor')
                lockdowns  = kwargs.get('lockdowns') # one off holidays

                # additional regressors what if
                regressors = kwargs.get('regressors') # dictionary pair of regressors and floats to multiply all the values of the regressor
                replace_regressors_values = kwargs.get('replace_regressors_values') # must be a nested disctionary 

                # additionnal regressors to e forcasted
                to_forecast = kwargs.get('to_forecast') # list like
                mode = kwargs.get('mode')
                
                # layout
                title  = kwargs.get('title')
                )_

## 1. INPUT MINIMI OLIGATORI

In questa sezione si descriveranno gli input munimi necessari alla funzione per svolgere il forecast.

### 1.1 DATA FRAME
Il data frame è il nome generale con la quale viene indicata la base dati sotto forma di tabella relazionale, in python questa viene ottenuta mediante la libreria Pandas.

#### 1.1.2 ANALISI MONOVARIATA
Il dataframe di input per il forecast monovariato ha bisogno di almeno 2 colonne: 
- **tempo:** 
la granularità del dataset dipende da questa colonna, la funzione accetta 2 tipi di granularità: giornaliera e mensile. 
Idealmente il nome della colonna dovrebbe essere 'ds', tuttavia nella funzione è predisposto un argomento per rinominarla conformemente a Prophet. 
Il dtype della colonnna dev'essere del tipo datetime, qualora la colonna non fosse di questo tipo di dato rimando alla documentazione di pandas al seguente link: 
https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
- **target:** il target è la colonna che vogliamo prevedere, idealmente il nome della colonna dovrebbe essere 'y', tuttavia, analogamente alla colonna di tempo, nella funzione è predisposto un argomento per rinominarla conformemente a Prophet. Il dtype della colonna deve essere necessariamente int o float, qualora non lo fosse: df['col'].astype(float). Il forecast deve avvenire per brand e per region/country.

#### 1.1.2 ANALISI MULTIVARIATA

Per l'analisi multivariata, oltre alle due colonne sopra menzionate per il forecast monovariato, sarà necessario aggiungere una colonna per ogni regressore che si voglia utilizzare per questo tipo di forecast. È di primaria importanza che i regressori riportino solo ed esclusivamente i valori relativi alla stagione dell'analisi in corso. Qualora vi fossero valori mancanti o non inerenti sarà necessario azzerarli mantenendo tuttavia la linearità storica invariata (no buchi temporati).

L'analisi multivariata può essere di tre tipi:
- **multivariata what if:** i valori futuri dei regressori possono essere modificati dall'analyst a seconda dei valori attesi. Questa sostituzione può avvennire sia imputando i valori relativi alla stagione precedente lasciandoli invariati o agendo su di essi con variazioni percentuali applicate all'intero periodo di forecast. Al contempo è statao predisposto un argomento della funzione per permettere la variazione del singolo valore di un quadrante (riga, colonna), approfondimento nel paragrafo KWARGS.
- **multivariata forecast**: qualora i regressori non siano dipendenti da fattori intrinseci e pertanto dipendenti dalle volontà del management sarà necessario prevedere gli stessi per permettere il forecast multivariato. Anche per questo tipo di analisi è stato predisposto un argomento della funzione per permettere ciò, approfondimento nel paragrafo KWARGS.
- **ibrido**: what if + forecast

#### 1.2 FORECAST FROM

Forecast from è il secondo _positional argument_ obbligatorio da inserire nella funzione. Esso va indicato sotto forma di stringa e deve essere coerente con il formato data utilizzato nella colonna di data stamp del data frame inputato. Come suggerisce il nome indica alla funzione la data dalla quale iniziare a fare il forecast, a seconda di quando questa ricada relativamente al data frame di riferimento la funzione darà in output casistiche differenti:
- **forecast_from < ultima data presente el data frame**: casistica di testing. Questo è il caso in cui si voglia testare l'accuratezza del modello. Ciò avviene confrontando i dati di output del forecast con quelli effettivamente verificatisi. Per quantificare il risultato vengono restituiti una serie di KPI che verranno descritti nella sezione 1.3.
- **forecast_from = al giorno/mese successivo l'ultimo riportato nel data frame**: casistica tipo di forecast operativo, gli output sarannno relativi l'intero periodo di riferimento compreso fra forecast_from e forecast_to. A seconda di quando ricada quest ultimo il periodo di paragone del forecast sarà la stagione conclusa più recente.
- **forecast_from > del giorno/mese successivo l'ultimo riportato nel data frame**: casistica tipo di forecast operativo, gli output sarannno unicamente relativi l'intero periodo di riferimento compreso fra forecast_from e forecast_to. A seconda di quando ricada quest ultimo il periodo di paragone del forecast sarà la stagione conclusa più recente. Dettaglio tecnico: dettato dal funzionamento di Prophet, il forecast reale avviene comunque a partire dal primo giorno non presente nell data frame di input, tuttavia i KPI e la data vizualisation in output saranno relativi al solo periodo di riferimento.

#### 1.3 FORECAST TO

L'ultimo positional argument da inserire obbligatoriamente è forecast_to. Come suggerisce il nome è la data fino alla quale verrà effettuato il forecast, essa sarà inclusa nell'intervallo. Analogamente a forecast_from anche in questo caso il formato deve essere coerente con la colonnna di data stamp. A seconda di quando questa cada rispettivamente al data frame inputato si apriranno scenari differenti:
- **forecast_to entro l'annno successivo del dataset in input**: qualora il forecast non superi la durata di un anno dall'ultima data riportata nel data frame di input, il forecast verrà paragonato con la stagione di riferimento precedente.
- **forecast_to oltre l'annno successivo del dataset in input**: qualora il forecast superi la durata di un anno dall'ultima data riportata nel data frame di input, il forecast verrà paragonato con la stagione di riferimento di due anni prima. Cioò avviene per assicurare maggiore uniformità nel confronto in modo da paragonare solo mensilità relative allo stesso macro periodo o stagione.

## 2. KWARGS

I KWARGS sono argomenti opzionali della funzione. A differenza dei positional argument non devono essere inseriti in un ordine preciso, tutttavia necessitano di essere preceduti dalla loro keyword identificativa.

#### 2.1 PREPROCESSING:

I seguenti KWARGS avranno effetto su data frame in input:
- **preprocess_from**: data dalla quale si vuole che il data frame in input inizi. _N.B. il formato data deve essere uguale a quello utilizzato nel data frame, la data indicata è inclusa nel data frame_
- **preprocess_to**: data dalla quale si vuole che il data frame in input finisca. _N.B. il formato data deve essere uguale a quello utilizzato nel data frame, la data indicata è inclusa nel data frame_
- **ds**: per indicare sotto forma di stringa il nome della colonna contenente il data stamp. Questa colonna verrà rinomita in 'ds' come richiesto da Prophet.
- **target**: per indicare sotto forma di stringa il nome della colonna oiettivo del forecast. Questa colonna verrà rinomita in 'y' come richiesto da Prophet.

#### 2.2 PROPHET

I seguenti KWARGS avranno effetto sui parametri di Prophet:
- **freq**: _def. 'M', dtype: str_ indica la frequeza/granularità del forecast e del data frame input. Può essere settato su 'M' per forecast mensile o su 'D' per forecast giornaliero
- **growth**: _def. 'linear', dtype: str_ indica il tipo di crescita del modello, 'linear' indica crescita lineare, 'logistic' logistica, per quest'ultima sarà necessario indicare anche _cap_ e _floor_ .
- **cap**: _dtype: int, float_ upper bound del forecast
- **floor**: _dtype: int, float_ lower bound del forecast
- **lockdowns**: _dtype: str_ nome del data frame contenente il/i periodo/i da segnalare come one-off-holiday: https://facebook.github.io/prophet/docs/handling_shocks.html#treating-covid-19-lockdowns-as-a-one-off-holidays

#### 2.3 REGRESSORI AGGIUNTIVI PER ANALISI WHAT IF:
- **regressors**: _dtype: dictionary es. {'reg1':1.0, 'reg2':1.15}_ dev'essere un dizionario contenente come chiavi i nomi del/i regressore/i esogeno/i da aggiungere al modello. Le chiavi del dizionario devono essere colonne del data frame di input. I valori corrispettivi sono i fattori da moltiplicare a ciascun valore relativo al periodo precedente del regressore di riferimento. Qualora forecast_to sia successivo ad un anno dall'ultima data riportata nel data frame i valori del regressore saranno quelli dello stesso intervallo temporale del forecast ma di due anni prima, altrimenti saranno riferiti all'anno precedente.
- **replace_regressors_values**: _dtype: nested dictionary es. {'reg1':{'data1':val1,'data2':val2}, 'reg1':{'data1':val1,'data2':val2}}_ permette di modificare i singoli valori relativi a quel regressore per quella data specifica. La chiave primaria del dizionaria dev'essere una stringa indicante il nome della colorra per la quale si vuole la sostituzione del valore. La data deve essere presente nel data frame e deve essere scritta nello stesso formato del data frame, altrimenti la sostituzione non avverrà. 

#### 2.4 REGRRSSORI AGGIUNTIVI DA FORECASTARE A LORO VOLTA

Qualora vi fossero regressori esogeni i quali valori futuri non possano essere stimati o imputati dall'utente sarà necessario che questi vengano a loro volta forecastati. Al momento il forecast avviene in maniera autoregressiva univariata per ciascuno di essi. Non è da escludre in futuro la possiilità di implementare la possiilità di forecast multivariato anche su di essi. La presenza di questo keword argument aggiungerà agli oputput il lineplot del regressore forecastato paragonato al dato reale dello stesso periodo di riferimento (qualora si tratti di caso test) o paragonato al dato di uno o due anni precedente a seconda che forecast_to sia di uno o più anni successivo all'ultima data riportata nel data frame in input.

- **to_forecast**: _dtype: list_ contenetne stringhe riportanti i nomi delle colonne relativi ai regressori  da forecastare che si vogliano aggiungere all'analisi multivariata.
- **mode**: _dtype: string_ mode può essere o 'additive' (default) o 'multiplicative', il kwargs avrà affetto su tutti i regressori.

#### 2.5 LAYOUT

Argomenti per modificare il layout dei grafici in output

- **title**: serve per modificare il titolo del barplot di comparazione fra forecast e dato reale

## 3. OUTPUT

Gli output della funzione variano a seconda del forecast specifico, tuttavia vi sono comunque dati e KPI che saranno sempre forniti in output.

### 3.1 OUTPUT IN COMUNE

- **tot forecast**: valore aggregato del target nel periodo di riferimento.
- **valore di paragone**: valore per avere un confronto fra un dato effettivamente osservato ed il  valore di forecast.
- **delta valore reale e di forecast**: espresso sia in termini reali che percentuali.
- **tabella pivot**: riportante il dato aggregato per mese in riga e per anno in colonna.
- **tabella di forecast**: riportante dato di forecast con lower e upper bound espresso sia in termini monetari che percentuali.
- **pivot visualizzata con plotly**
- **lineplot**: riportante il dato forecastato (linea sottile blu), fascia azzurro chiara di lower e upper bound, dato reale riportato con dot neri.
- **bar plot**: riportante il dato di forecast confrontato con dato reale. 
- **components plot**: riportante trent annnuali, mensili, settimanali e nel caso in cui vi siano apporto dei regressori.

### 3.2 OUTPUT WHAT IF

- **regressori aggiuntivi**: elenco regressori esogeni utilizzati nel forecast multivariato con relativi fattori di moltiplicazione.
- **data frame regressori esogeni**: data frame riportante tutti i valori utilizzati per il forecast multivariato.

### 3.3 OUTPUT MULTIVARIATO FORECAST

- **lineplot regressore**: lineplot per la visualizzazione del forecast univariato del regressore da utilizzarsi per il forecast multivariato.

### 3.4 OUTPUT TEST

- **The Root Mean Squared Error**: valore medio dello scostamento assoluto del forecast dal valore reale per il periodo oggetto, ottenuto elevando alla seconda l'errore. Questa misura penalizza scostamenti più elevati in valore assoluto.
- **The Mean Absolute Error**: valore medio dello scostamento assoluto del forecast dal valore reale per il periodo oggetto, ottenuto facendo la media del valore assoluto dell'errore. Questa misura non penalizza la magnitudine dell'errore. _N.B. minore sarà la differenza fra RMSE e MAE più vorrà dire che l'errore sarà di entità minore rispetto al valore totale del forecast sull'intero periodo_
- **The OVERALL MAPE**: media degli scostamenti assoluti del forecast dai valori reali di ogni punto di forecast. 
- **The OVERALL WAPE**: somma dei loss del singolo periodo sul valore reale dell'intero periodo.
- **The OVERALL WAPE MEAN**: media dei loss del singolo periodo sul valore reale dell'intero periodo.
- **regressori aggiuntivi**: elenco regressori esogeni utilizzati nel forecast multivariato con relativi fattori di moltiplicazione.
- **data frame regressori esogeni**: data frame riportante tutti i valori utilizzati per il forecast multivariato.
- **data frame valori reali**: riferito all'intero periodo oggetto di forecast.
- **data frame valori forecast**: riferito all'intero periodo oggetto di forecast con upper e lower bound.
- **final data frame**: di confronto riferito all'intero periodo oggetto di forecast con upper, lower bound e wape.
- **final data frame plotly**: di confronto riferito all'intero periodo oggetto di forecast con upper, lower bound e wape.
- **lineplot**: riportannte dato reale in rosa.
- **barplot**: riportannte dato reale in arancione.
- **components plot**: riportante trent annnuali, mensili, settimanali e nel caso in cui vi siano apporto dei regressori.





# IMPORTS

In [3]:
# IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from statsmodels.tools.eval_measures import rmse
from prophet.plot import plot_plotly, plot_components_plotly, add_changepoints_to_plot
import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error
from scipy import stats
from prophet.utilities import regressor_coefficients
from dateutil.relativedelta import relativedelta

# DEF FUNC

In [4]:
def make_forecast(df,
                  forecast_from, # INCLUDED, if forecast from > than the last day of the given df, data viz starts from that day
                  forecast_to,
                  freq='D', # INCLUDED
                  **kwargs):

    # IN COMUNE MONO/MULTI

    # KWARGS

    # Date
    preprocess_from = kwargs.get('preprocess_from')
    preprocess_to = kwargs.get('preprocess_to')

    # targets
    ds = kwargs.get('ds') # colonna del df da rinominare in ds nel caso in cui non lo fosse
    target = kwargs.get('target') # colonna del df da rinominare in y nel caso in cui non lo fosse
    # Prophet args
    #freq= kwargs.get('freq', 'M')
    growth = kwargs.get('growth', 'linear')
    cap  = kwargs.get('cap')
    floor  = kwargs.get('floor')
    lockdowns  = kwargs.get('lockdowns') # one off holidays
    # additional regressors what if
    regressors = kwargs.get('regressors') # dictionary pair of regressors and floats to multiply all the values of the regressor
    replace_regressors_values = kwargs.get('replace_regressors_values') # must be a nested disctionary 
    # additionnal regressors to e forcasted
    to_forecast = kwargs.get('to_forecast') # list like
    mode = kwargs.get('mode')
    # layout
    title  = kwargs.get('title')

    # da strinnga a datetime
    preprocess_from = pd.to_datetime(preprocess_from)
    preprocess_to = pd.to_datetime(preprocess_to)
    #start_date = pd.to_datetime(start_date)
    #end_date = pd.to_datetime(end_date)
    forecast_from = pd.to_datetime(forecast_from)
    forecast_to = pd.to_datetime(forecast_to)
    if forecast_from > (df['ds'].iloc[-1] + relativedelta(days=1)):
        forecast_from_oya = pd.to_datetime(pd.to_datetime(df['ds'].iloc[-1])-relativedelta(years=1))
        forecast_from_tya = pd.to_datetime(pd.to_datetime(df['ds'].iloc[-1])-relativedelta(years=2))
        forecast_from_oya0 = pd.to_datetime(forecast_from)-relativedelta(years=1)
        forecast_from_tya0 = pd.to_datetime(forecast_from)-relativedelta(years=2) 
    else:
        forecast_from_oya = pd.to_datetime(forecast_from)-relativedelta(years=1)
        forecast_from_tya = pd.to_datetime(forecast_from)-relativedelta(years=2)
        
    forecast_to_oya = pd.to_datetime(pd.to_datetime(forecast_to)-relativedelta(years=1))
    forecast_to_tya = pd.to_datetime(pd.to_datetime(forecast_to)-relativedelta(years=2))

    # DEF FUNCTIONS
    
    # mape
    def mape(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return str(np.mean(np.abs((y_true - y_pred))/y_true) * 100) + ' % of error'

    # wape
    def wape_mean(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return str((np.mean(np.abs(y_true-y_pred))/np.abs(np.sum(y_true)))* 100) + ' % of error'

    def wape(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return str((np.sum(np.abs(y_true-y_pred))/np.abs(np.sum(y_true)))* 100) + ' % of error'
    
    # replace regressors values
    def rrv(df, dict):
        for i,j in dict.items():
            for r in j:
                k = pd.to_datetime(r)
                df[i].loc[df['ds']==k] = j[r]
        return df

    # PREPROCESSING 
    df = df.rename({ds:'ds', target:'y'}, axis=1)

    if preprocess_to==None:
        preprocess_to = df['ds'].iloc[-1]
    else:
        df = df.loc[(df['ds'] >= preprocess_from) & (df['ds'] <= preprocess_to)]

    if forecast_from > (df['ds'].iloc[-1] + relativedelta(days=1)):
        # dates to create future forecast
        idx0 = pd.date_range(df['ds'].iloc[-1] + relativedelta(days=1), forecast_to).to_frame()
        future_days0 = len(idx0)
        idx0['month'] = idx0.index.month
        idx0['year'] = idx0.index.year
        idx0=idx0.groupby(['year','month'])[idx0.columns].max()
        n_months0 = len((idx0['month'])) # .unique()
        # dates for kpis and data viz
        idx = pd.date_range(forecast_from, forecast_to).to_frame()
        future_days = len(idx)
        idx['month'] = idx.index.month
        idx['year'] = idx.index.year
        idx=idx.groupby(['year','month'])[idx.columns].max()
        n_months = len((idx['month']))
    else:
        idx = pd.date_range(forecast_from, forecast_to).to_frame()
        future_days = len(idx)        
        idx['month'] = idx.index.month
        idx['year'] = idx.index.year
        idx=idx.groupby(['year','month'])[idx.columns].max()
        n_months = len((idx['month']))
        
    # aggiungo colonnna per crescita logistica
    df['floor']=floor
    df['cap']=cap

    # setting the model
    m = Prophet(
        growth=growth,
        holidays=lockdowns,
        #interval_width=0.95
    )
    # --------------------------------------------------                       ∆                       --------------------------------------------------
    # --------------------------------------------------                      ∆∆∆                      --------------------------------------------------
    # --------------------------------------------------                     ∆∆∆∆∆                     --------------------------------------------------
    # --------------------------------------------------                    ∆∆∆∆∆∆∆                    --------------------------------------------------
    # --------------------------------------------------                   ∆∆∆∆∆∆∆∆∆                   --------------------------------------------------
    # --------------------------------------------------                  ∆∆∆∆∆∆∆∆∆∆∆                  --------------------------------------------------
    # --------------------------------------------------                 ∆∆∆∆∆∆∆∆∆∆∆∆∆                 --------------------------------------------------
    # --------------------------------------------------                ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆                --------------------------------------------------
    # --------------------------------------------------               ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆               --------------------------------------------------
    # --------------------------------------------------              ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆              --------------------------------------------------
    # --------------------------------------------------             ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆             --------------------------------------------------
    # --------------------------------------------------            ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆            --------------------------------------------------
    # --------------------------------------------------           ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆           --------------------------------------------------
    # --------------------------------------------------          ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆          --------------------------------------------------
    # --------------------------------------------------         ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆         --------------------------------------------------
    # --------------------------------------------------        ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆        --------------------------------------------------
    # --------------------------------------------------       ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆       --------------------------------------------------
    # --------------------------------------------------      ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆      --------------------------------------------------
    # --------------------------------------------------     ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆     --------------------------------------------------
    # --------------------------------------------------    ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆    --------------------------------------------------
    # --------------------------------------------------   ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆   --------------------------------------------------
    # --------------------------------------------------  ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆  --------------------------------------------------    
    # -------------------------------------------------- ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆ FORECAST CASE ∆∆∆∆∆∆∆∆∆∆∆∆∆∆∆ --------------------------------------------------
    
    # forecast_to oltre storico dati stagione precente (no paragone) e forecst_from da oltre un mese da ultima data nel df
    if (forecast_to - relativedelta(years=1)) > df['ds'].iloc[-1] and forecast_from > df['ds'].iloc[-1] + relativedelta(days=1):
        # MULTIVARIATE
        # what if
        if regressors != None:
            for reg in regressors:
                m.add_regressor(reg,
                                mode=mode
                                )

        # forecasted 
        if to_forecast != None:
            def predictRegressor(df, x):
                df = df[["ds", x]]
                new_names = {x: "y"}
                df.rename(columns=new_names, inplace=True)

                m = Prophet()
                m.fit(df)

                #Create dates df for the future
                if freq=='D':
                    future = m.make_future_dataframe(periods = future_days0, freq=freq)
                elif freq=='M':
                    future = m.make_future_dataframe(periods = n_months0, freq=freq)                

                #Model predictions
                forecast = m.predict(future)
                
                # plot
                print('The forecasted regressor:')
                fig = plot_plotly(m, forecast)
                fig.show();
                print(('-----------------------------------------------------------------------------------------'))
                forecast.rename({'yhat':x}, axis=1, inplace=True)
                return forecast 

            for reg2 in to_forecast:
                m.add_regressor(reg2)

        # FITTING
        m.fit(df)

        # creating future df
        if freq=='D':
            future = m.make_future_dataframe(periods = future_days0, freq=freq)
        elif freq=='M':
            future = m.make_future_dataframe(periods = n_months0, freq=freq)

        future['floor']=floor
        future['cap']=cap

        # add additional regressors column
        if regressors != None:
            for reg, value in regressors.items():
                future = future.merge(df[[reg, 'ds']], on='ds',  how='left')
                if freq=='M':
                    future[reg].iloc[-n_months0:] = df[reg].loc[(df['ds']>forecast_from_tya)&(df['ds']<=forecast_to_tya)]*value
                elif freq=='D':
                    future[reg].iloc[-future_days0:] = df[reg].loc[(df['ds']>forecast_from_tya)&(df['ds']<=forecast_to_tya)]*value

        if replace_regressors_values != None:
            rrv(future, replace_regressors_values)

        if to_forecast != None:
            for reg3 in to_forecast:
                forecasted = predictRegressor(df, reg3)
                future[reg3] = forecasted[reg3]

        # predicting
        forecast = m.predict(future)

        # creating columns for groupby
        df['month'] = df['ds'].dt.month
        df['year'] = df['ds'].dt.year
        forecast['month'] = forecast['ds'].dt.month
        forecast['year'] = forecast['ds'].dt.year

        # creating df for results
        grouped_forecast = forecast\
            .loc[(forecast['ds']>=df['ds'].iloc[-1] + relativedelta(days=1)) & (forecast['ds']<=forecast_to)]\
            .groupby(['year', 'month'])['yhat', 'yhat_lower', 'yhat_upper', 'ds']\
            .agg({
                'yhat':'sum',
                'yhat_lower':'sum',
                'yhat_upper':'sum',
                'ds':'max'
                })\
                .reset_index()\
                .rename({'yhat':'y'}, axis=1)

        grouped_forecast['lower_bound_%'] = ((grouped_forecast['yhat_lower']-grouped_forecast['y'])/grouped_forecast['y'])*100
        grouped_forecast['upper_bound_%'] = ((grouped_forecast['yhat_upper']-grouped_forecast['y'])/grouped_forecast['y'])*100

        # KPI aggregate
        tot_value_by_month = df.groupby(['year', 'month'])['y'].sum().to_frame().reset_index()
        tot_prev_season = df['y'].loc[(df['ds']>forecast_from_tya0)&(df['ds']<=forecast_to_tya)].sum()
        delta = int(grouped_forecast['y'].iloc[-n_months:].sum()) - tot_prev_season
        delta_share = (delta/tot_prev_season)*100

        # merging
        final_forecast = grouped_forecast.append(tot_value_by_month)
        pivot = final_forecast.pivot_table(values = 'y', index = 'month', columns = 'year', fill_value=0, margins=False )
        pivot = pivot.astype(int)
        pivot = pivot.applymap('{:,.0f}'.format)

        #print
        print('-----------------------------------------------------------------------------------------')
        print('-----------------------------------------------------------------------------------------')
        print('The total predicted value of the UPCOMING season is {:,}'.format(int(grouped_forecast['y'].iloc[-n_months:].sum())))
        print('The total value of the PREVIOUS CLOSED season is {:,}'.format(int(tot_prev_season)))
        print('The differennce between the FORECASTED season and the PREVIOUS one is {:,}'.format(int(delta)))
        print('Expressed in % is {:,}'.format(round(delta_share, 2)))
        if regressors != None and len(regressors)>1:
            print('The parameters multiplied to the regressors are:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_tya])
        elif regressors != None and len(regressors)==1:
            print('The parameter multiplied to the regressors is:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_tya])
        print()
        print(pivot)
        print()
        print(grouped_forecast.iloc[-n_months:])
        #print(df.tail(10))
        
        table = go.Figure(data=[go.Table(
            header=dict(values=list(pivot.columns),
                    fill_color='paleturquoise',
                    align='left'),
            cells=dict(values=[pivot[c] for c in pivot.columns],
                fill_color='lavender',
                align='left'))
        ])
        # plot table 
        table.update_layout(width=1000, height=500)
        table.show()

        # plot graph 
        fig = plot_plotly(m, forecast)
        fig.show();

        # plot bars
        grouped_month_fig_forecast = px.bar(grouped_forecast.loc[grouped_forecast['ds']>=forecast_from], barmode='group', x='ds', y='y', color = 'month', title=title)
        grouped_month_fig_forecast.add_trace(go.Bar(
                  x = df['ds'].loc[(df['ds']>=forecast_from_tya0)&(df['ds']<=forecast_to_tya)],
                  y = df['y'].loc[(df['ds']>=forecast_from_tya0)&(df['ds']<=forecast_to_tya)],
                  name = 'fatturato stagione precedente'
                 ))
        grouped_month_fig_forecast.update_layout(legend_y=1.1, xaxis_type = 'category')
        grouped_month_fig_forecast.show();
        # plot compare

        # plot compoents
        if regressors != None:
            print(regressor_coefficients(m))
        m.plot_components(forecast)
           
    # forecast_to oltre storico dati stagione precente (no paragone) e forecst_from da entro un mese da ultima data nel df
    if (forecast_to - relativedelta(years=1)) > df['ds'].iloc[-1] and forecast_from <= df['ds'].iloc[-1] + relativedelta(days=1):
        # MULTIVARIATE
        # what if
        if regressors != None:
            for reg in regressors:
                m.add_regressor(reg,
                                mode=mode
                                )

        # forecasted 
        if to_forecast != None:
            def predictRegressor(df, x):
                df = df[["ds", x]]
                new_names = {x: "y"}
                df.rename(columns=new_names, inplace=True)

                m = Prophet()
                m.fit(df)

                #Create dates df for the future
                if freq=='D':
                    future = m.make_future_dataframe(periods = future_days, freq=freq)
                elif freq=='M':
                    future = m.make_future_dataframe(periods = n_months, freq=freq)                

                #Model predictions
                forecast = m.predict(future)

                # plot
                print('The forecasted regressor:')
                fig = plot_plotly(m, forecast)
                fig.show();
                print(('-----------------------------------------------------------------------------------------'))
                forecast.rename({'yhat':x}, axis=1, inplace=True)
                return forecast 

            for reg2 in to_forecast:
                m.add_regressor(reg2)

        # FITTING
        m.fit(df)

        # creating future df
        if freq=='D':
            future = m.make_future_dataframe(periods = future_days, freq=freq)
        elif freq=='M':
            future = m.make_future_dataframe(periods = n_months, freq=freq)

        future['floor']=floor
        future['cap']=cap

        # add additional regressors column
        if regressors != None:
            for reg, value in regressors.items():
                future = future.merge(df[[reg, 'ds']], on='ds',  how='left')
                if freq=='M':
                    future[reg].iloc[-n_months:] = df[reg].loc[(df['ds']>=forecast_from_tya)&(df['ds']<=forecast_to_tya)]*value
                elif freq=='D':
                    future[reg].iloc[-future_days:] = df[reg].loc[(df['ds']>=forecast_from_tya)&(df['ds']<=forecast_to_tya)]*value

        if replace_regressors_values != None:
            rrv(future, replace_regressors_values)

        if to_forecast != None:
            for reg3 in to_forecast:
                forecasted = predictRegressor(df, reg3)
                future[reg3] = forecasted[reg3]
        # predicting
        forecast = m.predict(future)

        # creating columns for groupby
        df['month'] = df['ds'].dt.month
        df['year'] = df['ds'].dt.year
        forecast['month'] = forecast['ds'].dt.month
        forecast['year'] = forecast['ds'].dt.year

        # creating df for results
        grouped_forecast = forecast\
            .loc[(forecast['ds']>=df['ds'].iloc[-1] + relativedelta(days=1)) & (forecast['ds']<=forecast_to)]\
            .groupby(['year', 'month'])['yhat', 'yhat_lower', 'yhat_upper', 'ds']\
            .agg({
                'yhat':'sum',
                'yhat_lower':'sum',
                'yhat_upper':'sum',
                'ds':'max'
                })\
                .reset_index()\
                .rename({'yhat':'y'}, axis=1)

        grouped_forecast['lower_bound_%'] = ((grouped_forecast['yhat_lower']-grouped_forecast['y'])/grouped_forecast['y'])*100
        grouped_forecast['upper_bound_%'] = ((grouped_forecast['yhat_upper']-grouped_forecast['y'])/grouped_forecast['y'])*100

        # KPI aggregate
        tot_value_by_month = df.groupby(['year', 'month'])['y'].sum().to_frame().reset_index()
        tot_prev_season = df['y'].loc[(df['ds']>forecast_from_tya)&(df['ds']<=forecast_to_tya)]
        tot_prev_season=tot_prev_season.iloc[-n_months:].sum()
        delta = int(grouped_forecast['y'].iloc[-n_months:].sum()) - tot_prev_season
        delta_share = (delta/tot_prev_season)*100

        # merging
        final_forecast = grouped_forecast.append(tot_value_by_month)
        pivot = final_forecast.pivot_table(values = 'y', index = 'month', columns = 'year', fill_value=0, margins=False )
        pivot = pivot.astype(int)
        pivot = pivot.applymap('{:,.0f}'.format)

        #print
        print('-----------------------------------------------------------------------------------------')
        print('-----------------------------------------------------------------------------------------')
        print('The total predicted value of the UPCOMING season is {:,}'.format(int(grouped_forecast['y'].iloc[-n_months:].sum())))
        print('The total value of the PREVIOUS CLOSED season is {:,}'.format(int(tot_prev_season)))
        print('The differennce between the FORECASTED season and the PREVIOUS one is {:,}'.format(int(delta)))
        print('Expressed in % is {:,}'.format(round(delta_share, 2)))
        if regressors != None and len(regressors)>1:
            print('The parameters multiplied to the regressors are:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_tya])
        elif regressors != None and len(regressors)==1:
            print('The parameter multiplied to the regressors is:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_tya])
        print()
        print(pivot)
        print()
        print(grouped_forecast.iloc[-n_months:])
        #print(df.tail(10))
        
        table = go.Figure(data=[go.Table(
            header=dict(values=list(pivot.columns),
                    fill_color='paleturquoise',
                    align='left'),
            cells=dict(values=[pivot[c] for c in pivot.columns],
                fill_color='lavender',
                align='left'))
        ])
        # plot table 
        table.update_layout(width=1000, height=500)
        table.show()

        # plot graph 
        fig = plot_plotly(m, forecast)
        fig.show();

        # plot bars
        grouped_month_fig_forecast = px.bar(grouped_forecast.loc[grouped_forecast['ds']>=forecast_from], barmode='group', x='ds', y='y', color = 'month', title=title)
        grouped_month_fig_forecast.add_trace(go.Bar(
                  x = df['ds'].loc[(df['ds']>=forecast_from_tya)&(df['ds']<=forecast_to_tya)],
                  y = df['y'].loc[(df['ds']>=forecast_from_tya)&(df['ds']<=forecast_to_tya)],
                  name = 'fatturato stagione precedente'
                 ))
        grouped_month_fig_forecast.update_layout(legend_y=1.1, xaxis_type = 'category')
        grouped_month_fig_forecast.show();
        # plot compare

        # plot compoents
        if regressors != None:
            print(regressor_coefficients(m))
        m.plot_components(forecast)
    
    # forecast_to entro storico dati stagione precente (si paragone) e forecst_from da oltre ultima data nel df
    if (forecast_to - relativedelta(years=1)) <= df['ds'].iloc[-1] and forecast_from > df['ds'].iloc[-1] + relativedelta(days=1):

        # MULTIVARIATE
        # what if
        if regressors != None:
            for reg in regressors:
                m.add_regressor(reg,
                                mode=mode
                                )

        # forecasted 
        if to_forecast != None:
            def predictRegressor(df, x):
                df = df[["ds", x]]
                new_names = {x: "y"}
                df.rename(columns=new_names, inplace=True)

                m = Prophet()
                m.fit(df)

                #Create dates df for the future
                if freq=='D':
                    future = m.make_future_dataframe(periods = future_days0, freq=freq)
                elif freq=='M':
                    future = m.make_future_dataframe(periods = n_months0, freq=freq)                

                #Model predictions
                forecast = m.predict(future)

                # plot
                print('The forecasted regressor:')
                fig = plot_plotly(m, forecast)
                fig.show();
                print(('-----------------------------------------------------------------------------------------'))
                forecast.rename({'yhat':x}, axis=1, inplace=True)
                return forecast 

            for reg2 in to_forecast:
                m.add_regressor(reg2)

        # FITTING
        m.fit(df)

        # creating future df
        if freq=='D':
            future = m.make_future_dataframe(periods = future_days0, freq=freq)
        elif freq=='M':
            future = m.make_future_dataframe(periods = n_months0, freq=freq)

        future['floor']=floor
        future['cap']=cap

        # add additional regressors column
        if regressors != None:
            for reg, value in regressors.items():
                future = future.merge(df[[reg, 'ds']], on='ds',  how='left')
                if freq=='M':
                    future[reg].iloc[-n_months0:] = df[reg].loc[(df['ds']>forecast_from_oya)&(df['ds']<=forecast_to_oya)]*value
                elif freq=='D':
                    future[reg].iloc[-future_days0:] = df[reg].loc[(df['ds']>forecast_from_oya)&(df['ds']<=forecast_to_oya)]*value

        if replace_regressors_values != None:
            rrv(future, replace_regressors_values)

        if to_forecast != None:
            for reg3 in to_forecast:
                forecasted = predictRegressor(df, reg3)
                future[reg3] = forecasted[reg3]
        # predicting
        forecast = m.predict(future)

        # creating columns for groupby
        df['month'] = df['ds'].dt.month
        df['year'] = df['ds'].dt.year
        forecast['month'] = forecast['ds'].dt.month
        forecast['year'] = forecast['ds'].dt.year

        # creating df for results
        grouped_forecast = forecast\
            .loc[(forecast['ds']>=df['ds'].iloc[-1] + relativedelta(days=1)) & (forecast['ds']<=forecast_to)]\
            .groupby(['year', 'month'])['yhat', 'yhat_lower', 'yhat_upper', 'ds']\
            .agg({
                'yhat':'sum',
                'yhat_lower':'sum',
                'yhat_upper':'sum',
                'ds':'max'
                })\
                .reset_index()\
                .rename({'yhat':'y'}, axis=1)

        grouped_forecast['lower_bound_%'] = ((grouped_forecast['yhat_lower']-grouped_forecast['y'])/grouped_forecast['y'])*100
        grouped_forecast['upper_bound_%'] = ((grouped_forecast['yhat_upper']-grouped_forecast['y'])/grouped_forecast['y'])*100

        # KPI aggregate
        tot_value_by_month = df.groupby(['year', 'month'])['y'].sum().to_frame().reset_index()
        tot_prev_season = df['y'].loc[(df['ds']>=forecast_from_oya)&(df['ds']<=forecast_to_oya)]
        tot_prev_season=tot_prev_season.iloc[-n_months:].sum()
        delta = int(grouped_forecast['y'].iloc[-n_months:].sum()) - tot_prev_season
        delta_share = (delta/tot_prev_season)*100

        # merging
        final_forecast = grouped_forecast.append(tot_value_by_month)
        pivot = final_forecast.pivot_table(values = 'y', index = 'month', columns = 'year', fill_value=0, margins=False )
        pivot = pivot.astype(int)
        pivot = pivot.applymap('{:,.0f}'.format)

        #print
        print('-----------------------------------------------------------------------------------------')
        print('-----------------------------------------------------------------------------------------')
        print('The total predicted value of the UPCOMING season is {:,}'.format(int(grouped_forecast['y'].iloc[-n_months:].sum())))
        print('The total value of the PREVIOUS season is {:,}'.format(int(tot_prev_season)))
        print('The differennce between the FORECASTED season and the PREVIOUS one is {:,}'.format(int(delta)))
        print('Expressed in % is {:,}'.format(round(delta_share, 2)))
        if regressors != None and len(regressors)>1:
            print('The parameters multiplied to the regressors are:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_oya])
        elif regressors != None and len(regressors)==1:
            print('The parameter multiplied to the regressors is:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_oya])
        print()
        print(pivot)
        print()
        print(grouped_forecast.iloc[-n_months:])
        #print(df.tail(10))
        
        table = go.Figure(data=[go.Table(
            header=dict(values=list(pivot.columns),
                    fill_color='paleturquoise',
                    align='left'),
            cells=dict(values=[pivot[c] for c in pivot.columns],
                fill_color='lavender',
                align='left'))
        ])
        # plot table 
        table.update_layout(width=1000, height=500)
        table.show()

        # plot graph 
        fig = plot_plotly(m, forecast)
        fig.show();

        # plot bars
        grouped_month_fig_forecast = px.bar(grouped_forecast.loc[grouped_forecast['ds']>=forecast_from], barmode='group', x='month', y='y', color = 'month', title=title)
        grouped_month_fig_forecast.add_trace(go.Bar(
                  x = df['month'].loc[(df['ds']>=forecast_from_oya0)&(df['ds']<=forecast_to_oya)],
                  y = df['y'].loc[(df['ds']>=forecast_from_oya0)&(df['ds']<=forecast_to_oya)],
                  name = 'fatturato stagione precedente'
                 ))
        grouped_month_fig_forecast.update_layout(legend_y=1.1, xaxis_type = 'category')
        grouped_month_fig_forecast.show();
        # plot compare

        # plot compoents
        if regressors != None:
            print(regressor_coefficients(m))
        m.plot_components(forecast)

            
    # forecast_to entro storico dati stagione precente (si paragone) e forecst_from da giorno dopo ultima data nel df
    if forecast_to - relativedelta(years=1) <= df['ds'].iloc[-1] and forecast_from == df['ds'].iloc[-1] + relativedelta(days=1):

        if regressors != None:
            for reg in regressors:
                m.add_regressor(reg,
                                mode=mode
                                )

        # forecasted 
        if to_forecast != None:
            def predictRegressor(df, x):
                df = df[["ds", x]]
                new_names = {x: "y"}
                df.rename(columns=new_names, inplace=True)

                m = Prophet()
                m.fit(df)

                #Create dates df for the future
                if freq=='D':
                    future = m.make_future_dataframe(periods = future_days, freq=freq)
                elif freq=='M':
                    future = m.make_future_dataframe(periods = n_months, freq=freq)                

                #Model predictions
                forecast = m.predict(future)

                # plot
                print('The forecasted regressor:')
                fig = plot_plotly(m, forecast)
                fig.show();
                print(('-----------------------------------------------------------------------------------------'))
                forecast.rename({'yhat':x}, axis=1, inplace=True)
                return forecast 

            for reg2 in to_forecast:
                m.add_regressor(reg2)

        # FITTING
        m.fit(df)

        # creating future df
        if freq=='D':
            future = m.make_future_dataframe(periods = future_days, freq=freq)
        elif freq=='M':
            future = m.make_future_dataframe(periods = n_months, freq=freq)

        future['floor']=floor
        future['cap']=cap


        # add additional regressors column
        if regressors != None:
            for reg, value in regressors.items():
                future = future.merge(df[[reg, 'ds']], on='ds',  how='left')
                if freq=='M':
                    future[reg].iloc[-n_months:] = df[reg].loc[(df['ds']>=forecast_from_oya)&(df['ds']<=forecast_to_oya)]*value
                elif freq=='D':
                    future[reg].iloc[-future_days:] = df[reg].loc[(df['ds']>=forecast_from_oya)&(df['ds']<=forecast_to_oya)]*value             
        
        if replace_regressors_values != None:
            rrv(future, replace_regressors_values)

        if to_forecast != None:
            for reg3 in to_forecast:
                forecasted = predictRegressor(df, reg3)
                future[reg3] = forecasted[reg3]
        # predicting
        forecast = m.predict(future)

        # creating columns for groupby
        df['month'] = df['ds'].dt.month
        df['year'] = df['ds'].dt.year
        forecast['month'] = forecast['ds'].dt.month
        forecast['year'] = forecast['ds'].dt.year

        # creating df for results
        grouped_forecast = forecast\
            .loc[(forecast['ds']>=df['ds'].iloc[-1] + relativedelta(days=1)) & (forecast['ds']<=forecast_to)]\
            .groupby(['year', 'month'])['yhat', 'yhat_lower', 'yhat_upper', 'ds']\
            .agg({
                'yhat':'sum',
                'yhat_lower':'sum',
                'yhat_upper':'sum',
                'ds':'max'
                })\
                .reset_index()\
                .rename({'yhat':'y'}, axis=1)

        grouped_forecast['lower_bound_%'] = ((grouped_forecast['yhat_lower']-grouped_forecast['y'])/grouped_forecast['y'])*100
        grouped_forecast['upper_bound_%'] = ((grouped_forecast['yhat_upper']-grouped_forecast['y'])/grouped_forecast['y'])*100

        # KPI aggregate
        tot_value_by_month = df.groupby(['year', 'month'])['y'].sum().to_frame().reset_index()
        tot_prev_season = df['y'].loc[(df['ds']>=forecast_from_oya)&(df['ds']<=forecast_to_oya)]
        tot_prev_season=tot_prev_season.iloc[-n_months:].sum()
        delta = int(grouped_forecast['y'].iloc[-n_months:].sum()) - tot_prev_season
        delta_share = (delta/tot_prev_season)*100

        # merging
        final_forecast = grouped_forecast.append(tot_value_by_month)
        pivot = final_forecast.pivot_table(values = 'y', index = 'month', columns = 'year', fill_value=0, margins=False )
        pivot = pivot.astype(int)
        pivot = pivot.applymap('{:,.0f}'.format)

        #print
        print('-----------------------------------------------------------------------------------------')
        print('-----------------------------------------------------------------------------------------')
        print('The total predicted value of the UPCOMING season is {:,}'.format(int(grouped_forecast['y'].iloc[-n_months:].sum())))
        print('The total value of the PREVIOUS season is {:,}'.format(int(tot_prev_season)))
        print('The differennce between the FORECASTED season and the PREVIOUS one is {:,}'.format(int(delta)))
        print('Expressed in % is {:,}'.format(round(delta_share, 2)))
        if regressors != None and len(regressors)>1:
            print('The parameters multiplied to the regressors are:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_oya])
        elif regressors != None and len(regressors)==1:
            print('The parameter multiplied to the regressors is:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from_oya])
        print()
        print(pivot)
        print()
        print(grouped_forecast.iloc[-n_months:])
        #print(df.tail(10))
        
        table = go.Figure(data=[go.Table(
            header=dict(values=list(pivot.columns),
                    fill_color='paleturquoise',
                    align='left'),
            cells=dict(values=[pivot[c] for c in pivot.columns],
                fill_color='lavender',
                align='left'))
        ])
        # plot table 
        table.update_layout(width=1000, height=500)
        table.show()

        # plot graph 
        fig = plot_plotly(m, forecast)
        fig.show();

        # plot bars
        grouped_month_fig_forecast = px.bar(grouped_forecast.loc[grouped_forecast['ds']>=forecast_from], barmode='group', x='month', y='y', color = 'month', title=title)
        grouped_month_fig_forecast.add_trace(go.Bar(
                  x = df['month'].loc[(df['ds']>=forecast_from_oya)&(df['ds']<=forecast_to_oya)],
                  y = df['y'].loc[(df['ds']>=forecast_from_oya)&(df['ds']<=forecast_to_oya)],
                  name = 'fatturato stagione precedente'
                 ))
        grouped_month_fig_forecast.update_layout(legend_y=1.1, xaxis_type = 'category')
        grouped_month_fig_forecast.show();
        # plot compare

        # plot compoents
        if regressors != None:
            print(regressor_coefficients(m))
        m.plot_components(forecast)
    # --------------------------------------------------                       ©                       --------------------------------------------------
    # --------------------------------------------------                      ©©©                      --------------------------------------------------
    # --------------------------------------------------                     ©©©©©                     --------------------------------------------------
    # --------------------------------------------------                    ©©©©©©©                    --------------------------------------------------
    # --------------------------------------------------                   ©©©©©©©©©                   --------------------------------------------------
    # --------------------------------------------------                  ©©©©©©©©©©©                  --------------------------------------------------
    # --------------------------------------------------                 ©©©©©©©©©©©©©                 --------------------------------------------------
    # --------------------------------------------------                ©©©©©©©©©©©©©©©                --------------------------------------------------
    # --------------------------------------------------               ©©©©©©©©©©©©©©©©©               --------------------------------------------------
    # --------------------------------------------------              ©©©©©©©©©©©©©©©©©©©              --------------------------------------------------
    # --------------------------------------------------             ©©©©©©©©©©©©©©©©©©©©©             --------------------------------------------------
    # --------------------------------------------------            ©©©©©©©©©©©©©©©©©©©©©©©            --------------------------------------------------
    # --------------------------------------------------           ©©©©©©©©©©©©©©©©©©©©©©©©©           --------------------------------------------------
    # --------------------------------------------------          ©©©©©©©©©©©©©©©©©©©©©©©©©©©          --------------------------------------------------
    # --------------------------------------------------         ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©         --------------------------------------------------
    # --------------------------------------------------        ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©        --------------------------------------------------
    # --------------------------------------------------       ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©       --------------------------------------------------
    # --------------------------------------------------      ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©      --------------------------------------------------
    # --------------------------------------------------     ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©     --------------------------------------------------
    # --------------------------------------------------    ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©    --------------------------------------------------
    # --------------------------------------------------   ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©   --------------------------------------------------
    # --------------------------------------------------  ©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©©  --------------------------------------------------    
    # -------------------------------------------------- ©©©©©©©©©©©©©©©©© TEST CASE ©©©©©©©©©©©©©©©©© --------------------------------------------------
    if forecast_to <= df['ds'].iloc[-1]:

        df_train = df.loc[df['ds']<forecast_from]
        df_test = df.loc[(df['ds']>=forecast_from) & (df['ds']<=forecast_to)]

        # MULTIVARIATE
        # what if
        if regressors != None:
            for reg in regressors:
                m.add_regressor(reg)

        # forecasted 
        if to_forecast != None:
            def predictRegressor(df, x):
                df = df_train[["ds", x]]
                new_names = {x: "y"}
                df.rename(columns=new_names, inplace=True)

                m = Prophet()
                m.fit(df_train)

                #Create dates df for the future
                if forecast_from > (df['ds'].iloc[-1] + relativedelta(days=1)):
                    if freq=='D':
                        future = m.make_future_dataframe(periods = future_days0, freq=freq)
                    elif freq=='M':
                        future = m.make_future_dataframe(periods = n_months0, freq=freq)
                else:
                    if freq=='D':
                        future = m.make_future_dataframe(periods = future_days, freq=freq)
                    elif freq=='M':
                        future = m.make_future_dataframe(periods = n_months, freq=freq)

                #Model predictions
                forecast = m.predict(future)
                forecast.rename({'yhat':x}, axis=1, inplace=True)
                return forecast
                
            def univariate_graph(y_true, y_pred):
                fig = px.line(y_true, x='ds', y="y")
                fig.add_trace(go.Line(mode="lines", x=y_pred['ds'], y=y_pred['yhat'], name="Prediction"))
                fig.update_layout(legend_y=1.1, xaxis_type = 'category')
                fig.show()

            for reg2 in to_forecast:
                m.add_regressor(reg2)
        
        m.fit(df_train)
        
        # creating future df
        if freq=='D':
            future = m.make_future_dataframe(periods = future_days, freq=freq)
        elif freq=='M':
            future = m.make_future_dataframe(periods = n_months, freq=freq)

        future['floor']=floor
        future['cap']=cap

        # add additional regressors column
        if regressors != None:
            for reg, value in regressors.items():
                future = future.merge(df_train[[reg, 'ds']], on='ds',  how='left')
                if freq=='M':
                    future[reg].iloc[-n_months:] = df_test[reg]*value
                if freq=='D':
                    future[reg].iloc[-future_days:] = df_test[reg]*value

        if to_forecast != None:
            for reg3 in to_forecast:
                forecasted = predictRegressor(df, reg3)
                future[reg3] = forecasted[reg3]
                future[reg3].loc[df['ds']<forecast_from] = df_train[reg3]

        

        # change values
        if replace_regressors_values != None:
            rrv(future, replace_regressors_values)

        #print(future)
        # predicting
        forecast = m.predict(future)

        # defining KPIs
        pred_total_value = forecast['yhat'].loc[(forecast['ds']>=forecast_from) & (forecast['ds']<=forecast_to)].sum()
        real_total_value = df['y'].loc[(df['ds']>=forecast_from) & (df['ds']<=forecast_to)].sum()
        delta = pred_total_value - real_total_value

        # creating columns for groupby
        df['month'] = df['ds'].dt.month
        df['year'] = df['ds'].dt.year
        forecast['month'] = forecast['ds'].dt.month
        forecast['year'] = forecast['ds'].dt.year

        # creating single dfs
        tot_value_by_month = df.loc[(df['ds']>=forecast_from) & (df['ds']<=forecast_to)].groupby(['year', 'month'])['y'].sum().to_frame().reset_index()
        grouped_forecast = forecast.loc[(forecast['ds']>=forecast_from) & (forecast['ds']<=forecast_to)].groupby(['year', 'month'])['yhat', 'yhat_lower', 'yhat_upper', 'ds'].agg({'yhat':'sum','yhat_lower':'sum','yhat_upper':'sum','ds':'max'}).reset_index()
        grouped_forecast['lower_bound_%'] = ((grouped_forecast['yhat_lower']-grouped_forecast['yhat'])/grouped_forecast['yhat'])*100
        grouped_forecast['upper_bound_%'] = ((grouped_forecast['yhat_upper']-grouped_forecast['yhat'])/grouped_forecast['yhat'])*100

        # creating final df
        final_df = tot_value_by_month.merge(grouped_forecast, on = ['year', 'month'], how = 'inner')
        final_df['dlt(pred-real)'] = final_df['yhat'] - final_df['y']
        final_df['%'] = (final_df['dlt(pred-real)']/final_df['y'])*100;
        final_df['wape %'] = final_df['dlt(pred-real)']/np.sum(final_df['y'])*100
        
        # final_df2 è il df formattato a stringa per visualizzare meglio i numeri
        final_df2 = final_df.astype(int)
        final_df2 = final_df.applymap('{:,.0f}'.format)
        
        ma_pe = mape(final_df['y'], final_df['yhat'])
        wa_pe = wape(final_df['y'], final_df['yhat'])
        wa_pe_mean = wape_mean(final_df['y'], final_df['yhat'])
    
        # printouts
        print('-----------------------------------------------------------------------------------------');
        print('-----------------------------------------------------------------------------------------');
        print('The total predicted value of the season is {:,} €'.format(int(pred_total_value)));
        print('The total real value of the season is {:,} €'.format(int(real_total_value)));
        print('The difference between the real value and the predicted one is {:,} €'.format(int(delta)));
        print('Expressed in % is: {:,}  '.format(int((delta/real_total_value)*100)))
        #print('The Mean Squared Error is {:,} %'.format(mean_squared_error(final_df['y'], final_df['yhat'])))
        print('The Root Mean Squared Error is {:,} %'.format(np.sqrt(mean_squared_error(final_df['y'], final_df['yhat']))))
        #print('The Mean Squared Log Error is {:,} %'.format(mean_squared_log_error(final_df['y'], final_df['yhat'])))
        print('The Mean Absolute Error is {:,} %'.format(mean_absolute_error(final_df['y'], final_df['yhat'])))
        print('The OVERALL MAPE is {}  %'.format((ma_pe)));
        print('The OVERALL WAPE  is {}  %'.format((wa_pe)));
        print('The OVERALL WAPE MEAN is {}  %'.format((wa_pe_mean)));
        if regressors != None and len(regressors)>1:
            print('The parameters multiplied to the regressors are:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from])
        elif regressors != None and len(regressors)==1:
            print('The parameter multiplied to the regressors is:{}'.format(regressors))
            print(future.loc[future['ds']>=forecast_from])        
        print('-----------------------------------------------------------------------------------------');
        print('-----------------------------------------------------------------------------------------');
        print('The REAL monthly bdg is:');
        print(tot_value_by_month[-n_months:])
        print('-----------------------------------------------------------------------------------------');
        print('The PREDICTED monthly bdg is:');
        print(grouped_forecast[-n_months:])
        print('-----------------------------------------------------------------------------------------');
        print('FINAL DF')
        print(final_df[-n_months:]);
        print('-----------------------------------------------------------------------------------------');
        print('-----------------------------------------------------------------------------------------')
        
        table = go.Figure(data=[go.Table(
        header=dict(values=list(final_df2[['year','month','y','yhat','dlt(pred-real)','%','wape %']].columns),
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[final_df2.year, final_df2.month, final_df2['y'], final_df2.yhat, final_df2['dlt(pred-real)'], final_df2['%'], final_df2['wape %']],
                fill_color='lavender',
                align='left'))
        ])
        table.update_layout(width=1000, height=500)
        table.show()

        # plotting overall forecast
        fig = plot_plotly(m, forecast)
        fig.add_trace(go.Line(mode="lines", x=df['ds'].loc[(df['ds']>=forecast_from) & (df['ds']<=forecast_to)], y=df['y'].loc[(df['ds']>=forecast_from) & (df['ds']<=forecast_to)], name="Actual price"))
        fig.show();

        # plotting overall
        overall = px.bar(final_df[-n_months:], x='month', y=['yhat'], color = 'month', title=title, 
        barmode='group'
        )
        overall.add_trace(go.Bar(x = final_df['month'][-n_months:], y = final_df['y'][-n_months:], name = 'real', 
        ))
        overall.update_layout(legend_y=1.1, xaxis_type = 'category')
        overall.show()

        if regressors != None:
            m.plot_components(forecast)
        # plotting components
        m.plot_components(forecast)

        if to_forecast != None:
            if freq=='M':
                univariate_graph(final_df[['ds','y']][-n_months:], final_df[['ds','yhat']][-n_months:])
            elif freq=='D':
                univariate_graph(final_df[['ds','y']][-future_days:], final_df[['ds','yhat']][-future_days:])
